# Agent Simulations V3

What is new in this notebook:
- A generic `RoundRobinSimulator` for running a simulation of an agent dialogue in a round-robin fashion. It takes an `agents` list and a `leader_idx` as input. The agents speak in a round-robin fashion, where the agent at index `i` speaks to the agent at index `(i+1) % len(agents)`. The agent at index `leader_idx` speaks first.

Comments:
- This uses the `ChatOpenAI` model, which expects chat roles to either be `assistant` or `user`. This restricts us to only two agents. Therefore, even though `RoundRobinSimulator` can support multiple agents, it really only makes sense to use it for two agents because the AIMessage that agent `i` sends is received as a HumanMessage by agent `i+1`.
- If all we want is a simulator for two agents right now, the `RoundRobinSimulator` is a complete MVP.

This notebook uses [CAMEL](https://www.camel-ai.org/) as a use case.

## Import LangChain related modules 

In [1]:
from collections import OrderedDict
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Define base classes and CAMEL classes

In [2]:
class Entity:
    def reset(self):
        raise NotImplementedError
        
    def step(self):
        raise NotImplementedError

class CAMELAgent(Entity):

    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.message_history = [self.system_message]

    def reset(self, message=None) -> None:
        if message is not None:
            self.message_history.append(message)
        return self.message_history

    def _update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.message_history.append(message)
        return self.message_history

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        
        messages = self._update_messages(input_message)
        output_message = self.model(messages)
        self._update_messages(output_message)

        return output_message


In [3]:
class BaseSimulator:
    
    def reset(self):
        raise NotImplementedError 
    
    def step(self):
        raise NotImplementedError 
    
class RoundRobinSimulator(BaseSimulator):
    def __init__(self, agents: List, leader_idx: int):
        self.agents = agents
        self.leader_idx = leader_idx
        
    def reset(self, message_from_leader):
        """
        The conversation begins with a message from the leader
        """
        # initialize the leader
        msg_from_leader = AIMessage(content=message_from_leader)
        leader = self.agents[self.leader_idx]
        leader.reset(msg_from_leader)
        
        # initialize the other agents
        for i in range(len(self.agents)-1):
            idx = (self.leader_idx + i + 1) % len(self.agents)
            self.agents[idx].reset()
            
        return HumanMessage(content=message_from_leader)
    
    def step(self, msg_from_leader):
        messages = dict()
        message = msg_from_leader
        
        # the last agent to speak will be the leader
        for i in range(len(self.agents)):
            idx = (self.leader_idx + i + 1) % len(self.agents)

            message = self.agents[idx].step(message)
            message = HumanMessage(content=message.content)
            
            messages[idx] = message
            
        return messages

## Setup roles and task for role-playing

In [4]:
assistant_role_name = "Python Programmer"
user_role_name = "Stock Trader"
task = "Develop a trading bot for the stock market"
word_limit = 50 # word limit for task brainstorming

## Create a task specify agent for brainstorming and get the specified task

In [5]:
task_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(content=
        f"""Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
        Please make it more specific. Be creative and imaginative.
        Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
        )
]
specified_task = ChatOpenAI(temperature=1.0)(task_specifier_prompt).content

## Create system messages for AI assistant and AI user

In [6]:
def generate_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    
    # the prompt that the user gives to the assistant
    assistant_inception_prompt = (
    f"""Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles! Never instruct me!
    We share a common interest in collaborating to successfully complete a task.
    You must help me to complete the task.
    Here is the task: {task} 
    Never forget our task!
    I must instruct you based on your expertise and my needs to complete the task.

    I must give you one instruction at a time.
    You must write a specific solution that appropriately completes the requested instruction.
    You must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.
    Do not add anything else other than your solution to my instruction.
    You are never supposed to ask me any questions you only answer questions.
    You are never supposed to reply with a flake solution. Explain your solutions.
    Your solution must be declarative sentences and simple present tense.
    Unless I say the task is completed, you should always start with:

    Solution: <YOUR_SOLUTION>

    <YOUR_SOLUTION> should be specific and provide preferable implementations and examples for task-solving.
    Always end <YOUR_SOLUTION> with: Next request."""
    )

    # the prompt that the assistant gives to the user
    user_inception_prompt = (
    f"""Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always instruct me.
    We share a common interest in collaborating to successfully complete a task.
    I must help you to complete the task.
    Here is the task: {task} 
    Never forget our task!
    You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

    1. Instruct with a necessary input:
    Instruction: <YOUR_INSTRUCTION>
    Input: <YOUR_INPUT>

    2. Instruct without any input:
    Instruction: <YOUR_INSTRUCTION>
    Input: None

    The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

    You must give me one instruction at a time.
    I must write a response that appropriately completes the requested instruction.
    I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
    You should instruct me not ask me questions.
    Now you must start to instruct me using the two ways described above.
    Do not add anything else other than your instruction and the optional corresponding input!
    Keep giving me instructions and necessary inputs until you think the task is completed.
    When the task is completed, you must only reply with a single word <CAMEL_TASK_DONE>.
    Never say <CAMEL_TASK_DONE> unless my responses have solved your task."""
    )
    
    return SystemMessage(content=assistant_inception_prompt), SystemMessage(content=user_inception_prompt) 


## Create AI assistant agent and AI user agent

In [7]:
assistant_sys_msg, user_sys_msg = generate_sys_msgs(assistant_role_name, user_role_name, specified_task)
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(temperature=0.2))
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(temperature=0.2))

## Main Loop

In [8]:
print(f"Original task prompt:\n{task}\n")
print(f"Specified task prompt:\n{specified_task}\n")

max_iters = 2
n = 0

roles = [assistant_role_name, user_role_name]  # ordered
agents = [assistant_agent, user_agent]
leader_idx = 0  # the assistant speaks first because it is index 0
simulator = SequentialSimulator(agents=agents, leader_idx=leader_idx)

message_text_from_assistant = (
            "Now start to give me instructions one by one. "
            "Only reply with Instruction and Input.")
message_from_assistant = simulator.reset(message_text_from_assistant)

while n < max_iters:
    messages = simulator.step(message_from_assistant)
    for idx, message in messages.items():
        print(f"AI ({roles[idx]}):\n\n{message.content}\n\n")

        if roles[idx] == user_role_name and "<CAMEL_TASK_DONE>" in message.content:
            break
        
    message_from_assistant = messages[leader_idx]
        
    n += 1

Original task prompt:
Develop a trading bot for the stock market

Specified task prompt:
Develop a customized trading bot that uses machine learning algorithms and real-time data analysis to predict stock market trends, track market news and sentiments, and execute profitable trades automatically for the company's diversified portfolio, while ensuring risk management and compliance with relevant regulations.

AI (Stock Trader):

Instruction: Install necessary libraries for machine learning and data analysis.
Input: None.


AI (Python Programmer):

Solution: To install necessary libraries for machine learning and data analysis, we can use the following commands in the terminal:

- For machine learning libraries: 
    - pip install scikit-learn
    - pip install tensorflow
    - pip install keras

- For data analysis libraries:
    - pip install pandas
    - pip install numpy
    - pip install matplotlib

Next request.


AI (Stock Trader):

Instruction: Import the necessary libraries in 